In [0]:
import os
from google.colab import drive
drive_root = '/content/drive'
if not os.path.exists('/content/drive'):
  drive.mount(drive_root, force_remount=True)

In [0]:
!apt install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.


In [0]:
root = '/content/drive/My Drive/bin-movie'

In [0]:
!ls -lh '{root}'

total 473M
drwx------ 2 root root 4.0K Jun 15 09:08 bin_imgs
-rw------- 1 root root 6.1M Jun 15 10:09 bin_imgs.zip
drwx------ 2 root root 4.0K Jun 15 09:03 imgs
-rw------- 1 root root 171M Jun 15 10:09 imgs.zip
-rw------- 1 root root 297M Apr 26 14:07 input.mp4


In [0]:
!mkdir -p 'imgs'

In [0]:
!ffmpeg -i '{root}/input.mp4' -vf fps=1,scale=400:225 'imgs/out%05d.png'

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [0]:
!zip -rq ./imgs.zip ./imgs

In [0]:
!du -sh imgs.zip

171M	imgs.zip


In [0]:
!pip install opencv-python tqdm joblib

In [0]:
import os
from pathlib import Path
import cv2
import math
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed

In [0]:
def get_image(gray, a, b):
    return 255 - cv2.Canny(gray, a, b)

def get_gray(fpath):
    img = cv2.imread(fpath)
    height, width = img.shape[:2]
    height, width = 255, 400
    img = cv2.resize(img, (width, height))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGBA)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return gray

In [0]:
imgs_root = Path(f'imgs')

In [0]:
a, b = 10, 40

In [0]:
os.makedirs(f'bin_imgs', exist_ok=True)

In [0]:
def convert(inpath, outpath):
    img = get_image(get_gray(inpath), 10, 40)
    cv2.imwrite(outpath, img)

In [0]:
params = []
for dirname, _, filenames in os.walk(imgs_root):
    filenames = [x for x in filenames if x.endswith('.png')]
    for f in tqdm(filenames):
        inpath = str(imgs_root / f)
        outpath = f'bin_imgs/{f}'
        params.append((inpath, outpath))

100%|██████████| 1523/1523 [00:00<00:00, 124691.10it/s]


In [0]:
_ = Parallel(n_jobs=-1, verbose=0)(delayed(convert)(a, b) for a, b in tqdm(params))

100%|██████████| 1523/1523 [00:08<00:00, 182.28it/s]


In [0]:
!ls 'bin_imgs' | wc -l

1523


In [0]:
!zip -rq bin_imgs.zip bin_imgs

In [0]:
!du -sh 'bin_imgs.zip'

13M	bin_imgs.zip


In [0]:
!cp imgs.zip '{root}/'
!cp bin_imgs.zip '{root}/'

In [0]:
!ls -lh '{root}' 